In [ ]:
!pip install wandb
!wandb login

## Gradient Descent From Scratch

In [ ]:
import numpy as np
import torch

### Training Data

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [ ]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [ ]:
# Create the weights and biases
w = torch.randn((2, 3), requires_grad=True)
b = torch.randn(2, requires_grad=True)

print(w)
print(b)

tensor([[ 0.5601, -0.3552, -0.5379],
        [-0.3850,  0.6177, -1.1442]], requires_grad=True)
tensor([-0.8196,  0.7227], requires_grad=True)


In [ ]:
def model(x):
  return x @ w.t() + b

model(inputs)

tensor([[ -6.8643, -35.1994],
        [-15.5391, -53.1865],
        [-30.8927, -16.3685],
        [ 21.1318, -54.3238],
        [-33.9307, -46.6400]], grad_fn=<AddBackward0>)

In [ ]:
# MSE Loss - Mean Square Error
def mse(preds, targets):
  diff = preds - targets
  return torch.sum(diff*diff) / diff.numel()

loss = mse(model(inputs), targets)
loss

tensor(14741.8281, grad_fn=<DivBackward0>)

In [ ]:
loss.backward()

RuntimeError: ignored

In [ ]:
loss.backward()

with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5
  w.grad.zero_()
  b.grad.zero_()

mse(model(inputs), targets)

tensor(2710.2029, grad_fn=<DivBackward0>)

In [ ]:
# Train for multiple epochs
for i in range(1000):
  preds = model(inputs)
  loss = mse(preds, targets)
  loss.backward()

  with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [ ]:
preds = model(inputs)

print(preds)
print(targets)
print(mse(preds, targets))

tensor([[ 57.0530,  70.3429],
        [ 82.2778, 100.6489],
        [118.7100, 132.9531],
        [ 21.0953,  37.0110],
        [101.9036, 119.1432]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])
tensor(0.5110, grad_fn=<DivBackward0>)


#USING THE PYTORCH LIBRARY

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

## Training Data

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

## Dataset and DataLoader

In [ ]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [ ]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [ ]:
for xb, yb in train_dl:
  print(xb)
  print(yb)
  break

tensor([[102.,  43.,  37.],
        [ 91.,  87.,  65.],
        [ 73.,  67.,  43.],
        [ 68.,  97.,  70.],
        [ 87., 134.,  58.]])
tensor([[ 22.,  37.],
        [ 80., 102.],
        [ 56.,  70.],
        [102., 120.],
        [119., 133.]])


## nn.Linear

In [ ]:
# Define the model
model = nn.Linear(3,2)

print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.1777,  0.0923, -0.4564],
        [-0.0064,  0.5677, -0.0191]], requires_grad=True)
Parameter containing:
tensor([-0.2730,  0.3262], requires_grad=True)


In [ ]:
print(list(model.parameters()))

[Parameter containing:
tensor([[ 0.1777,  0.0923, -0.4564],
        [-0.0064,  0.5677, -0.0191]], requires_grad=True), Parameter containing:
tensor([-0.2730,  0.3262], requires_grad=True)]


In [ ]:
# Generate predictions
pred = model(inputs)
pred

tensor([[ -0.7412,  37.0737],
        [ -5.1890,  48.4795],
        [  1.0841,  74.7327],
        [  4.9371,  23.3781],
        [-11.0997,  53.0474],
        [ -0.6557,  36.4996],
        [ -5.7377,  47.8928],
        [  0.8054,  74.7072],
        [  4.8516,  23.9522],
        [-11.7339,  53.0348],
        [ -1.2899,  36.4870],
        [ -5.1035,  47.9054],
        [  1.6328,  75.3194],
        [  5.5713,  23.3907],
        [-11.1852,  53.6215]], grad_fn=<AddmmBackward0>)

## Loss Function

In [ ]:
loss_fn = F.mse_loss

loss = loss_fn(model(inputs), targets)
loss

tensor(4958.7705, grad_fn=<MseLossBackward0>)

## Optimizer

In [ ]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

## Training Loop

In [ ]:
# Utility function to train the model

def fit(num_epochs, model, loss_fn, opt, train_dl):
  for epoch in range(num_epochs):
    for xb, yb in train_dl:
      pred = model(xb)
      loss = loss_fn(pred, yb)
      loss.backward()
      opt.step()
      opt.zero_grad()
    if (epoch+1) % 10 == 0:
      # print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item()}")
      print('Epoch [{}/{}] Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


In [ ]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100] Loss: 29.6115
Epoch [20/100] Loss: 27.1717
Epoch [30/100] Loss: 13.9419
Epoch [40/100] Loss: 21.2669
Epoch [50/100] Loss: 28.7550
Epoch [60/100] Loss: 15.6598
Epoch [70/100] Loss: 23.6873
Epoch [80/100] Loss: 8.9403
Epoch [90/100] Loss: 11.1339
Epoch [100/100] Loss: 14.0685


In [ ]:
pred = model(inputs)
pred

tensor([[ 57.1674,  70.7143],
        [ 79.5165,  98.2914],
        [123.3093, 137.4258],
        [ 22.2770,  39.0169],
        [ 96.5972, 113.9173],
        [ 55.8730,  69.5870],
        [ 78.9130,  97.9740],
        [123.3555, 137.7989],
        [ 23.5714,  40.1442],
        [ 97.2882, 114.7273],
        [ 56.5639,  70.3970],
        [ 78.2220,  97.1640],
        [123.9128, 137.7432],
        [ 21.5860,  38.2069],
        [ 97.8917, 115.0447]], grad_fn=<AddmmBackward0>)

In [ ]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [ ]:
loss_fn(pred, targets)

tensor(12.8749, grad_fn=<MseLossBackward0>)

## Building a Feedforward Neural Network from our Linear Model

In [ ]:
model2 = nn.Sequential(
    nn.Linear(3, 4),
    nn.ReLU(),
    nn.Linear(4, 2)
)

opt2 = torch.optim.SGD(model2.parameters(), lr=1e-7)

In [ ]:
fit(10000, model2, loss_fn, opt2, train_dl)

Epoch [10/10000] Loss: 9596.5059
Epoch [20/10000] Loss: 6596.1357
Epoch [30/10000] Loss: 7169.8384
Epoch [40/10000] Loss: 4291.6240
Epoch [50/10000] Loss: 6532.5894
Epoch [60/10000] Loss: 6468.0889
Epoch [70/10000] Loss: 5323.3232
Epoch [80/10000] Loss: 4118.2627
Epoch [90/10000] Loss: 5662.6284
Epoch [100/10000] Loss: 4849.6343
Epoch [110/10000] Loss: 3194.0813
Epoch [120/10000] Loss: 2234.1064
Epoch [130/10000] Loss: 5260.9492
Epoch [140/10000] Loss: 3953.0669
Epoch [150/10000] Loss: 2502.5886
Epoch [160/10000] Loss: 2969.1201
Epoch [170/10000] Loss: 2932.8374
Epoch [180/10000] Loss: 2575.2141
Epoch [190/10000] Loss: 2260.4727
Epoch [200/10000] Loss: 562.6266
Epoch [210/10000] Loss: 3372.8926
Epoch [220/10000] Loss: 1273.5802
Epoch [230/10000] Loss: 1575.4902
Epoch [240/10000] Loss: 968.1326
Epoch [250/10000] Loss: 2152.1296
Epoch [260/10000] Loss: 1593.4084
Epoch [270/10000] Loss: 783.8942
Epoch [280/10000] Loss: 473.3074
Epoch [290/10000] Loss: 1022.4686
Epoch [300/10000] Loss: 107

## Blog Notes - Robotics Ignite
#### Series - Machine Learning to Reinforment Learning
#### Series - Matering ROS and Gazebo!
#### NLP, NLU, Everything Communication for AI!



1.   Getting staeted with Jupyter Notebooks! Your companion tool on the ML Journey

    // getting free jupyter notes
    
    // setting up jupyter notes on your machine - Linux and Windows

    // Mastering Markdown
    
    ---



2.   PyTorch Basics! Your companion library on the ML Journey

    // setting up pytorch - colab, kaggle, local

    // PyTorch Basics. Autograd

    ---

3.  Linear Regression and Gradient Descent from Scratch! Your first ML Model




In [ ]:
## Include in your blog demonstrations of fitting the hyperparamters
## I had issues with the hyperparamters
## I had issues with the non-Linear activation function, and then variations 
## in the learning rate towards convergence. Reducing the learning rate resulted
## in better convergeance